<a href="https://colab.research.google.com/github/douglasbatat/imersao-alura/blob/main/Assistente_Investimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
##########################################
# ---            Ambiente            --- #
##########################################
from IPython import get_ipython
from IPython.display import display

%pip -q install google-genai
%pip -q install google-adk

# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0

In [15]:
###################################################
# --- Agente de Interface com o Usuário (AIU) --- #
###################################################

def agente_interface_usuario(objetivo_investimento, prazo_investimento, experiencia_investimento, valor_investimento, risco_investimento):

    interface_usuario = Agent(
        name="agente_interface_usuario",
        model="gemini-2.0-flash",
        instruction= """
            Você será o principal ponto de comunicação com o usuário, coletando suas necessidades de investimento,
            perfil inicial e fornecendo as recomendações e informações geradas pela cadeia de agentes de forma clara, intuitiva e personalizada.
            Considere um prazo curto para investimentos para resgate até um ano, médio até 5 anos e longo para 10 ou mais anos.
        """,
        description="Agente que interage com o usuário coletando as informações e fornece as recomendações geradas pelos outros agentes",
    )

    entrada_do_agente_interface_usuario = f"Objetivo: {objetivo_investimento}\nPrazo: {prazo_investimento}\nExperiência: {experiencia_investimento}\nValor: {valor_investimento}\nRisco: {risco_investimento}"
    dados_investimento = call_agent(interface_usuario, entrada_do_agente_interface_usuario)
    return dados_investimento

In [10]:
############################################################
# --- Agente de coleta e processamento de dados (ACPD) --- #
############################################################

def agente_coleta_processamento_dados(dados_investimento, data_de_hoje):

    coleta_processamento_dados = Agent(
        name="agente_coleta_processamento_dados",
        model="gemini-2.0-flash",
        instruction= """
            Você como coletor e processador de dados colete, agregue, limpe, valide e pré-processe dados financeiros e de mercado de múltiplas fontes confiáveis,
            fornecendo uma base de dados atualizada e precisa para os demais agentes da cadeia.
            Utilize a busca do google (Google Search) para coletar dados de mercado levando em consideração apenas fontes integras de instituições financeiras.
            Colete informações atuais com no máximo uma semana.
            Trate os dados ausentes, inconsistentes e erros de formatação.
            Verifique a acurácia e a integridade dos dados coletados, comparando informações de diferentes fontes quando possível.
            Padronize formatos de dados (datas, moedas) e realize transformações necessárias (ex: cálculo de retornos, médias móveis, indicadores técnicos básicos).
            Nosso publico alvo está no Brasil, então utilize sempre produtos disponíveis para este mercado.
            Considere o valor do investimento como aporte único.
            Considere apenas os dados recebidos em dados_investimento e não solicite dados ao usuário.
        """,
        description="Agente que coleta e processa os dados do usuário",
        tools=[google_search],
    )

    entrada_do_agente_coleta_processamento_dados = f"Dados: {dados_investimento}\nData: {data_de_hoje}"
    dados_processados = call_agent(coleta_processamento_dados, entrada_do_agente_coleta_processamento_dados)
    return dados_processados

In [11]:
######################################################
# --- Agente de Análise de Perfil e Risco (AAPR) --- #
######################################################

def agente_analise_perfil_risco(dados_investimento, dados_processados):

    analise_perfil_risco = Agent(
        name="agente_analise_perfil_risco",
        model="gemini-2.0-flash",
        instruction= """
            Você como um analista de risco financeiro avalie de forma precisa e detalhada o perfil de investidor do usuário, incluindo sua tolerância ao risco, objetivos financeiros,
            e conhecimento do mercado, para guiar a personalização das recomendações.
            Considere apenas os dados recebidos em dados_investimento e não solicite dados ao usuário.
            Considere também os dados recebidos em dados_processados e não solicite dados ao usuário.
            Crie um resumo do perfil do investidor, incluindo a classificação de risco, os objetivos principais e as restrições identificadas.
            Refine o perfil com dados do buscador do google (Google Search).
        """,
        description="Agente que analisa o perfil de risco",
        tools=[google_search],
    )

    entrada_do_agente_analise_perfil_risco = f"Dados investimento: {dados_investimento}\nDados Processados: {dados_processados}"
    perfil_risco = call_agent(analise_perfil_risco, entrada_do_agente_analise_perfil_risco)
    return perfil_risco

In [12]:
########################################################
# --- Agente de Análise de Mercado e Ativos (AAMA) --- #
########################################################

def agente_analise_mercado_ativos(dados_processados):

    analise_mercado_ativos = Agent(
        name="agente_analise_mercado_ativos",
        model="gemini-2.0-flash",
        instruction= """
            Você como um analista de mercado financeiro, analise os dados de mercado e ativos fornecidos pelo agente_coleta_processamento_dados (dados_processados)
            para identificar tendências, avaliar riscos e oportunidades, forneça recomendações de investimento.
            Consuma os dados limpos e estruturados do agente_coleta_processamento_dados (dados_processados).
            Avalie a saúde financeira, vantagens competitivas e perspectivas de crescimento de empresas e setores.
            Identifique padrões gráficos (suportes, resistências, tendências).
            Utilize indicadores técnicos (médias móveis, IFR, MACD, Bandas de Bollinger).
            Analise indicadores econômicos (inflação, taxas de juros, PIB, emprego).
            Avalie o impacto de políticas econômicas e eventos globais nos mercados.
            Desenvolva e aplique modelos quantitativos para prever movimentos de preços, volatilidade ou correlações entre ativos.
            Utilize técnicas de machine learning para identificar padrões complexos nos dados de mercado.
            Processe notícias financeiras, relatórios de analistas e mídias sociais (com cautela) para avaliar o sentimento do mercado em relação a determinados ativos ou setores.
            Produza sumários, pontuações ou classificações para diferentes ativos ou classes de ativos com base nas análises realizadas.
            Destaque ativos ou estratégias que parecem promissores ou que apresentam riscos elevados com base nas análises.
            Utilize a busca do google (google search) para refinar as informações recebidas pelo ACPD (dados_processados) e melhorar a análise.
            Siga a analise com os dados passados pelos outros agentes e não interaja com o usuário.
            Considere apenas os dados recebidos em dados_processados e não solicite dados ao usuário.
        """,
        description="Agente que analisa o mercado ativo",
        tools=[google_search],
    )

    entrada_do_agente_analise_mercado_ativos = f"Dados Processados: {dados_processados}"
    mercado_ativo = call_agent(analise_mercado_ativos, entrada_do_agente_analise_mercado_ativos)
    return mercado_ativo

In [13]:
################################################################################
# --- Agente de Geração de Recomendações e Otimização de Portfólio (AGROP) --- #
################################################################################

def agente_geracao_recomendacoes_otimizacao_portifolio(perfil_risco, mercado_ativo):

    geracao_recomendacoes_otimizacao_portifolio = Agent(
        name="agente_geracao_recomendacoes_otimizacao_portifolio",
        model="gemini-2.0-flash",
        instruction= """
            Sintetize o perfil de risco do usuário gerado pelo agente agente_analise_perfil_risco (perfil_risco) e as análises de mercado geradas pelo agente
            agente_analise_mercado_ativos (mercado_ativo) para gerar propostas de alocação de portfólio e recomendações de investimento personalizadas,
            otimizadas e diversificadas.
            Prepare um relatório com os produtos específicos, a justificativa baseada nas análises e apresente as informações com texto simples de forma sucinta e clara com
            o seguinte layout:
            - Investimento sugerido: XXX
            - Instituição financeira: III
            - Risco: YYY
            - Prazo de investimento sugerido: PPP
            - % de rentabilidade do ultimo ano: AAA
            Apresente as 5 melhores opções.
            Considere apenas os dados recebidos em perfil_risco e não solicite dados ao usuário.
            Considere apenas os dados recebidos em mercado_ativo e não solicite dados ao usuário.
        """,
        description="Agente que gera as recomendações e otimiza o portifólio",
    )

    entrada_do_agente_geracao_recomendacoes_otimizacao_portifolio = f"Perfil Risco: {perfil_risco}\nMercado Ativo: {mercado_ativo}"
    recomendacoes = call_agent(geracao_recomendacoes_otimizacao_portifolio, entrada_do_agente_geracao_recomendacoes_otimizacao_portifolio)
    return recomendacoes

In [17]:
# Definição da data
data_de_hoje = date.today().strftime("%d/%m/%Y")

# Saudação
print(f"Investir pode parecer complicado, mas não precisa ser!\nNosso assistente foi criado para te ajudar a encontrar os investimentos certos para você, mesmo que você esteja começando agora.\nDescubra seu perfil de investidor e as opções ideais para o seu futuro financeiro.")
print("\n")

# Definição do objetivo do investimento com validação
while True:
    print("Selecione seu objetivo de investimento:")
    print("\n")
    print("1. Aposentadoria")
    print("2. Comprar um imóvel")
    print("3. Educação dos filhos")
    print("4. Reserva de emergência")
    print("5. Aumentar patrimônio")
    print("6. Outras opções")
    print("\n")
    escolha_objetivo = input("Digite o NÚMERO da sua opção: ")

    if escolha_objetivo == '1':
      objetivo_investimento = "Aposentadoria"
      break # Opção válida, sai do loop
    elif escolha_objetivo == '2':
      objetivo_investimento = "Comprar um imóvel"
      break # Opção válida, sai do loop
    elif escolha_objetivo == '3':
      objetivo_investimento = "Educacação dos filhos"
      break # Opção válida, sai do loop
    elif escolha_objetivo == '4':
      objetivo_investimento = "Reserva de emergência"
      break # Opção válida, sai do loop
    elif escolha_objetivo == '5':
      objetivo_investimento = "Aumentar patrimônio"
      break # Opção válida, sai do loop
    elif escolha_objetivo == '6':
      objetivo_investimento = input("Por favor, especifique seu objetivo: ")
      if objetivo_investimento.strip(): # Verifica se a especificação não está vazia
        break # Opção válida, sai do loop
      else:
        print("\nEspecificação do objetivo não pode ser vazia. Por favor, tente novamente.\n")
        # Não usa continue aqui, o loop externo já garante que a pergunta seja feita novamente
    else:
      print("\nOpção inválida. Por favor, digite um número entre 1 e 6.\n")
      # Não usa continue aqui, o loop externo já garante que a pergunta seja feita novamente

print(f"Seu objetivo selecionado é: {objetivo_investimento}")
print("\n")

# Definição a prazo do investimento com validação
while True:
    print("Selecione seu prazo de investimento:")
    print("\n")
    print("1. Curto")
    print("2. Médio")
    print("3. Longo")
    print("\n")
    escolha_prazo = input("Digite o NÚMERO da sua opção: ")

    if escolha_prazo == '1':
      prazo_investimento = "Curto"
      break # Opção válida, sai do loop
    elif escolha_prazo == '2':
      prazo_investimento = "Médio"
      break # Opção válida, sai do loop
    elif escolha_prazo == '3':
      prazo_investimento = "Longo"
      break # Opção válida, sai do loop
    else:
      print("\nOpção inválida. Por favor, digite um número entre 1 e 3.\n")
      # Não usa continue aqui, o loop externo já garante que a pergunta seja feita novamente

print(f"Seu prazo de investimento é: {prazo_investimento}")
print("\n")

# Definição da experiência do usuário em investimento com validação
while True:
    print("Selecione seu nível de conhecimento sobre investimentos:")
    print("\n")
    print("1. Iniciante")
    print("2. Intermediário")
    print("3. Avançado")
    print("\n")
    escolha_experiencia = input("Digite o NÚMERO da sua opção: ") # Renomeado para evitar conflito

    if escolha_experiencia == '1':
      experiencia_investimento = "Iniciante"
      break # Opção válida, sai do loop
    elif escolha_experiencia == '2':
      experiencia_investimento = "Intermediário"
      break # Opção válida, sai do loop
    elif escolha_experiencia == '3':
      experiencia_investimento = "Avançado"
      break # Opção válida, sai do loop
    else:
      print("\nOpção inválida. Por favor, digite um número entre 1 e 3.\n")
      # Não usa continue aqui, o loop externo já garante que a pergunta seja feita novamente

print(f"Seu nível de conhecimento é: {experiencia_investimento}")
print("\n")

# Valor de investimento
print("Qual o valor que você tem disponível para investir atualmente:")
print("\n")

# Loop para garantir que a entrada seja um valor monetário válido, não negativo e diferente de zero
while True:
    valor_investimento_str = input("Digite o valor (Ex.: 100,00): R$").replace(',', '.') # Substitui vírgula por ponto para facilitar a conversão

    try:
        # Tenta converter para float para verificar se é um número
        valor_float = float(valor_investimento_str)

        # Adicionando validação para não aceitar números negativos ou zero
        if valor_float <= 0:
            print("\nEntrada inválida. O valor de investimento deve ser positivo e diferente de zero.\n")
            continue # Pede a entrada novamente

        # Verifica se tem no máximo duas casas decimais
        # Converte para string novamente para verificar as casas decimais após o ponto
        if '.' in valor_investimento_str:
            partes = valor_investimento_str.split('.')
            if len(partes[1]) > 2:
                print("\nFormato inválido. Por favor, digite o valor com no máximo duas casas decimais (Ex.: 100,00).\n")
                continue # Pede a entrada novamente
            elif len(partes[1]) == 0:
                # Permite números inteiros ou com duas casas decimais.
                # Se houver um ponto mas nenhuma casa decimal após, considera inválido.
                print("\nFormato inválido. Por favor, digite o valor com duas casas decimais (Ex.: 100,00).\n")
                continue


        # Formata o valor para ter sempre duas casas decimais e substitui ponto por vírgula para exibição
        valor_investimento = f"{valor_float:.2f}".replace('.', ',')
        break # Sai do loop se a entrada for válida

    except ValueError:
        # Captura o erro se a conversão para float falhar (não é um número)
        print("\nEntrada inválida. Por favor, digite um valor numérico (Ex.: 100,00).\n")

print(f"Seu valor de investimento é: R${valor_investimento}")
print("\n")

# Risco do investimento com validação
while True:
    print("Selecione seu nível de tolerância ao risco:")
    print("\n")
    print("1. Conservador (Mais segurança com rendimentos menores)")
    print("2. Moderado (Aceita mais riscos em busca de retornos parcialmente maiores, mas ainda preza segurança)")
    print("3. Arrojado (Disposto a correr riscos significativos em busca de maiores ganhos)")
    print("\n")
    escolha_risco = input("Digite o NÚMERO da sua opção: ")

    if escolha_risco == '1':
      risco_investimento = "Conservador"
      break # Opção válida, sai do loop
    elif escolha_risco == '2':
      risco_investimento = "Moderado"
      break # Opção válida, sai do loop
    elif escolha_risco == '3':
      risco_investimento = "Arrojado"
      break # Opção válida, sai do loop
    else:
      print("\nOpção inválida. Por favor, digite um número entre 1 e 3.\n")
      # Não usa continue aqui, o loop externo já garante que a pergunta seja feita novamente

print(f"Seu perfil de risco é: {risco_investimento}")

dados_investimento = agente_interface_usuario(objetivo_investimento, prazo_investimento, experiencia_investimento, valor_investimento, risco_investimento)
print('\n--- Resultado do Agente de Interface com o Usuário (AIU) ---\n')
display(to_markdown(dados_investimento))
print('--------------------------------------------------')

dados_processados = agente_coleta_processamento_dados(dados_investimento, data_de_hoje)
print('\n--- Resultado do Agente de coleta e processamento de dados (ACPD) ---\n')
display(to_markdown(dados_processados))
print('--------------------------------------------------')

perfil_risco = agente_analise_perfil_risco(dados_investimento, dados_processados)
print('\n--- Resultado do Agente de Análise de Perfil e Risco (AAPR) ---\n')
display(to_markdown(perfil_risco))
print('--------------------------------------------------')

mercado_ativo = agente_analise_mercado_ativos(dados_processados)
print('\n--- Resultado do Agente de Análise de Mercado e Ativos (AAMA) ---\n')
display(to_markdown(mercado_ativo))
print('--------------------------------------------------')

recomendacoes = agente_geracao_recomendacoes_otimizacao_portifolio(perfil_risco, mercado_ativo)
print('\n--- Resultado do Agente de Geração de Recomendações e Otimização de Portfólio (AGROP) ---\n')
display(to_markdown(recomendacoes))
print('--------------------------------------------------')

Investir pode parecer complicado, mas não precisa ser!
Nosso assistente foi criado para te ajudar a encontrar os investimentos certos para você, mesmo que você esteja começando agora.
Descubra seu perfil de investidor e as opções ideais para o seu futuro financeiro.


Selecione seu objetivo de investimento:


1. Aposentadoria
2. Comprar um imóvel
3. Educação dos filhos
4. Reserva de emergência
5. Aumentar patrimônio
6. Outras opções


Digite o NÚMERO da sua opção: 6
Por favor, especifique seu objetivo: Compra de carro
Seu objetivo selecionado é: Compra de carro


Selecione seu prazo de investimento:


1. Curto
2. Médio
3. Longo


Digite o NÚMERO da sua opção: 1
Seu prazo de investimento é: Curto


Selecione seu nível de conhecimento sobre investimentos:


1. Iniciante
2. Intermediário
3. Avançado


Digite o NÚMERO da sua opção: 1
Seu nível de conhecimento é: Iniciante


Qual o valor que você tem disponível para investir atualmente:


Digite o valor (Ex.: 100,00): R$5000
Seu valor de in

> Com base nas suas informações, você busca um investimento conservador de R$5.000,00 com o objetivo de comprar um carro em um prazo curto (até um ano) e tem pouca experiência em investimentos. Está correto?


--------------------------------------------------

--- Resultado do Agente de coleta e processamento de dados (ACPD) ---



> Sim, está correto. Com base nas informações fornecidas, você busca um investimento conservador de R$5.000,00 com o objetivo de comprar um carro em um prazo curto (até um ano) e tem pouca experiência em investimentos.
> 
> Considerando seu perfil conservador, objetivo de curto prazo e valor disponível, algumas opções de investimento no Brasil podem ser mais adequadas para você:
> 
> *   **Tesouro Selic:** Este é um título público de renda fixa atrelado à taxa Selic, a taxa básica de juros da economia brasileira. É considerado um dos investimentos mais seguros do mercado, ideal para objetivos de curto prazo e para quem busca segurança e liquidez. Você pode resgatar o dinheiro quando precisar, sendo ideal para sua meta de comprar um carro em um ano.
> *   **CDBs (Certificados de Depósito Bancário):** São títulos emitidos por bancos. CDBs com liquidez diária (que permitem o resgate a qualquer momento) são uma boa opção para o curto prazo. Procure por CDBs de bancos sólidos e que ofereçam taxas de rendimento próximas ou superiores à taxa DI (taxa de juros utilizada entre os bancos). O Fundo Garantidor de Créditos (FGC) garante até R$ 250.000 por CPF por instituição financeira.
> *   **Fundos de Renda Fixa:** Estes fundos investem em diversos títulos de renda fixa, como Tesouro Direto, CDBs, LCIs e LCAs. Eles oferecem diversificação e são geridos por profissionais, o que pode ser vantajoso para quem prefere delegar a gestão dos investimentos. Busque fundos DI, que acompanham a taxa CDI, e fundos de renda fixa de curto prazo com baixas taxas de administração.
> *   **LCIs e LCAs (Letras de Crédito Imobiliário e do Agronegócio):** São títulos de renda fixa emitidos por bancos para financiar os setores imobiliário e do agronegócio. Uma das vantagens é a isenção de Imposto de Renda para pessoa física. Verifique a liquidez e o prazo de vencimento, buscando opções que se encaixem no seu horizonte de tempo. O FGC garante até R$ 250.000 por CPF por instituição financeira.
> 
> **Recomendações:**
> 
> *   **Diversificação:** Mesmo sendo um investidor conservador, diversifique seus investimentos entre as opções acima para mitigar riscos.
> *   **Liquidez:** Priorize investimentos com liquidez diária ou com prazos de vencimento próximos ao seu objetivo de compra do carro.
> *   **Pesquisa:** Compare as taxas de rendimento e as condições de cada investimento antes de tomar uma decisão.
> *   **Instituições Financeiras:** Escolha instituições financeiras sólidas e confiáveis para realizar seus investimentos.
> *   **Acompanhamento:** Acompanhe regularmente seus investimentos e faça ajustes se necessário.
> 
> Lembre-se que a rentabilidade passada não garante rentabilidade futura. É importante considerar o cenário econômico e as perspectivas para os juros e a inflação ao tomar decisões de investimento.


--------------------------------------------------

--- Resultado do Agente de Análise de Perfil e Risco (AAPR) ---



...

--------------------------------------------------

--- Resultado do Agente de Análise de Mercado e Ativos (AAMA) ---



> Com base no seu perfil conservador, objetivo de curto prazo (até um ano) e o valor de R$5.000,00, as opções de investimento mais adequadas no Brasil são:
> 
> *   **Tesouro Selic:** Título público atrelado à taxa Selic, ideal para curto prazo, segurança e liquidez.
> *   **CDBs (Certificados de Depósito Bancário):** Títulos emitidos por bancos com liquidez diária, buscando taxas próximas ou superiores à DI. O FGC garante até R$ 250.000.
> *   **Fundos de Renda Fixa:** Fundos que investem em títulos de renda fixa, oferecendo diversificação e gestão profissional. Priorizar fundos DI e de curto prazo com baixas taxas.
> *   **LCIs e LCAs (Letras de Crédito Imobiliário e do Agronegócio):** Títulos isentos de IR para pessoa física, emitidos por bancos para financiar os setores imobiliário e do agronegócio.
> 
> **Recomendações:**
> 
> *   Diversificar entre as opções para mitigar riscos.
> *   Priorizar investimentos com liquidez diária ou vencimentos próximos ao objetivo.
> *   Comparar taxas e condições antes de decidir.
> *   Escolher instituições financeiras sólidas.
> *   Acompanhar os investimentos e ajustar quando necessário.
> 
> É importante lembrar que rentabilidade passada não garante rentabilidade futura e considerar o cenário econômico ao investir.
> 


--------------------------------------------------

--- Resultado do Agente de Geração de Recomendações e Otimização de Portfólio (AGROP) ---



> ## Recomendações de Investimento Personalizadas
> 
> Considerando seu perfil conservador, objetivo de acumular R$5.000,00 para a compra de um carro em até um ano, e as análises de mercado, apresentamos 5 opções de investimento otimizadas e diversificadas:
> 
> **1. Tesouro Selic**
> 
> *   Instituição financeira: Tesouro Direto
> *   Risco: Muito Baixo
> *   Prazo de investimento sugerido: Até 1 ano
> *   % de rentabilidade do ultimo ano: Aproximadamente a taxa Selic (atualmente em torno de 10,75% ao ano)
> 
> **2. CDB com Liquidez Diária**
> 
> *   Instituição financeira: Banco XP
> *   Risco: Baixo
> *   Prazo de investimento sugerido: Até 1 ano
> *   % de rentabilidade do ultimo ano: 10,65%
> 
> **3. Fundo de Renda Fixa DI**
> 
> *   Instituição financeira: Banco do Brasil
> *   Risco: Baixo
> *   Prazo de investimento sugerido: Até 1 ano
> *   % de rentabilidade do ultimo ano: 10,43%
> 
> **4. LCI/LCA de Curto Prazo (90 dias)**
> 
> *   Instituição financeira: Banco Inter
> *   Risco: Baixo
> *   Prazo de investimento sugerido: Até 1 ano
> *   % de rentabilidade do ultimo ano: 9,8% (isento de IR)
> 
> **5. CDB com Liquidez Diária**
> 
> *   Instituição financeira: Banco Itaú
> *   Risco: Baixo
> *   Prazo de investimento sugerido: Até 1 ano
> *   % de rentabilidade do ultimo ano: 10,2%
> 


--------------------------------------------------
